In [1]:
import tensorflow as tf
import os
import numpy as np

USE_GPU = True

if USE_GPU:
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    gpu_devices = tf.config.experimental.list_physical_devices("GPU")
    for device in gpu_devices:
        tf.config.experimental.set_memory_growth(device, True)
else:
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
    
from tensorflow.keras.preprocessing.image import ImageDataGenerator   

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train = x_train / 255.0
x_test = x_test / 255.0

datagen = ImageDataGenerator(rotation_range=90,
                 width_shift_range=0.1, height_shift_range=0.1,
                 horizontal_flip=True) 

# replica = 10

# x_train_len = x_train.shape[0]
# x_train_image_shape = x_train.shape[1:]

# x_train_argumented = np.empty((x_train_len*replica,*x_train_image_shape))
# y_train_argumented = np.empty((x_train_len*replica,*y_train.shape[1:]))

# for index in range(replica):
#     print(index)
#     x_train_argumented[x_train_len*index:x_train_len*(index+1)] = next(datagen.flow(x_train, batch_size=x_train_len))
#     y_train_argumented[x_train_len*index:x_train_len*(index+1)] = y_train
    
y_train_one_hot = tf.squeeze(tf.one_hot(y_train, 10), axis=1)
y_test_one_hot = tf.squeeze(tf.one_hot(y_test, 10), axis=1)

In [2]:
def learning_rate_scheduler(
    epoch, learning_rate, schedule_list=None, exponent=0.1, warm_up=True, warm_up_epoch=10
):
    step = 0
    if warm_up and epoch < warm_up_epoch:
        new_learning_rate = learning_rate * ((epoch + 1) / warm_up_epoch)
    else:
        if schedule_list is None:
            schedule_list = [25, 50, 100, 175, 250, 400]
        for step, target_epoch in enumerate(schedule_list):
            if target_epoch > epoch:
                break
            else:
                continue
        new_learning_rate = learning_rate * (exponent ** (step))

    return new_learning_rate

In [3]:
from gan_module.model.build_model import build_classifier
from tensorflow.keras.optimizers import Nadam
input_img_shape = (32, 32, 3)
optimizer = Nadam(1e-3)
num_class = 10
loss_function = tf.keras.losses.BinaryCrossentropy(label_smoothing=0.1)
model = build_classifier(
    input_img_shape=input_img_shape,
    classfier_power=16,
    depth=4,
    num_class=num_class,
)

In [4]:
from tensorflow.keras.callbacks import History 
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import Adam

check_point = ModelCheckpoint(
    "./temp/weights_{epoch:02d}_{loss:.4f}.hdf5", 
    monitor='val_loss', 
    verbose=0, 
    save_best_only=False, 
    save_weights_only=True, 
    mode='min')

reduceLROnPlat = ReduceLROnPlateau(
      monitor="val_loss",
      factor=0.1,
      patience=20,
      verbose=0,
      mode="auto",
      min_delta=0.0001,
      cooldown=5,
      min_lr=1e-7)
reduceLROnPlat = tf.keras.callbacks.ReduceLROnPlateau(
        monitor="val_loss", factor=0.5,
        patience=5, verbose=1
    )
lr_scheduler = LearningRateScheduler(learning_rate_scheduler)
history = History()
model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

In [5]:
nb_epoch = 400
batch_size = 32
model.fit(x=x_train,
          y=y_train_one_hot,
            epochs=nb_epoch,
            batch_size=batch_size,
            validation_split=0.1,
            shuffle = True,
            callbacks = [check_point, reduceLROnPlat, history])

Epoch 1/400
1407/1407 [==============================] - 47s 33ms/step - loss: 0.3974 - accuracy: 0.3583 - val_loss: 0.4279 - val_accuracy: 0.2520
Epoch 2/400
1407/1407 [==============================] - 45s 32ms/step - loss: 0.3356 - accuracy: 0.5377 - val_loss: 0.3456 - val_accuracy: 0.5010
Epoch 3/400
1407/1407 [==============================] - 46s 33ms/step - loss: 0.3206 - accuracy: 0.6056 - val_loss: 0.4023 - val_accuracy: 0.3522
Epoch 4/400
1407/1407 [==============================] - 46s 32ms/step - loss: 0.3130 - accuracy: 0.6413 - val_loss: 0.3299 - val_accuracy: 0.5762
Epoch 5/400
1407/1407 [==============================] - 46s 33ms/step - loss: 0.3080 - accuracy: 0.6624 - val_loss: 0.3173 - val_accuracy: 0.6292
Epoch 6/400
1407/1407 [==============================] - 47s 33ms/step - loss: 0.3051 - accuracy: 0.6767 - val_loss: 0.3474 - val_accuracy: 0.5114
Epoch 7/400
1407/1407 [==============================] - 46s 32ms/step - loss: 0.3022 - accuracy: 0.6905 - val_loss: 0

Epoch 50/400
1407/1407 [==============================] - 47s 33ms/step - loss: 0.2432 - accuracy: 0.9069 - val_loss: 0.2922 - val_accuracy: 0.7534
Epoch 51/400
1407/1407 [==============================] - 47s 33ms/step - loss: 0.2429 - accuracy: 0.9079 - val_loss: 0.2916 - val_accuracy: 0.7548
Epoch 52/400
1407/1407 [==============================] - 47s 33ms/step - loss: 0.2432 - accuracy: 0.9069 - val_loss: 0.2925 - val_accuracy: 0.7540
Epoch 53/400
1407/1407 [==============================] - 45s 32ms/step - loss: 0.2432 - accuracy: 0.9072 - val_loss: 0.2919 - val_accuracy: 0.7540
Epoch 54/400
1407/1407 [==============================] - ETA: 0s - loss: 0.2426 - accuracy: 0.9093
Epoch 00054: ReduceLROnPlateau reducing learning rate to 7.812500371073838e-06.
1407/1407 [==============================] - 45s 32ms/step - loss: 0.2426 - accuracy: 0.9093 - val_loss: 0.2926 - val_accuracy: 0.7526
Epoch 55/400
1407/1407 [==============================] - 45s 32ms/step - loss: 0.2427 - accu

Epoch 95/400
1407/1407 [==============================] - 45s 32ms/step - loss: 0.2413 - accuracy: 0.9132 - val_loss: 0.2929 - val_accuracy: 0.7516 loss: 0.2405 - accu - ETA: 26s - loss - E - ETA: 21s - loss: - ETA: 17s - loss: 0.2411 - - ETA: 16s - loss: 0.2409 - accuracy: 0. - ETA: 16s - loss: 0.
Epoch 96/400
1407/1407 [==============================] - 45s 32ms/step - loss: 0.2422 - accuracy: 0.9091 - val_loss: 0.2929 - val_accuracy: 0.7510
Epoch 97/400
1407/1407 [==============================] - 46s 32ms/step - loss: 0.2415 - accuracy: 0.9120 - val_loss: 0.2928 - val_accuracy: 0.7526
Epoch 98/400
1407/1407 [==============================] - 45s 32ms/step - loss: 0.2415 - accuracy: 0.9128 - val_loss: 0.2930 - val_accuracy: 0.7526
Epoch 99/400
1407/1407 [==============================] - ETA: 0s - loss: 0.2412 - accuracy: 0.9133
Epoch 00099: ReduceLROnPlateau reducing learning rate to 1.525878978725359e-08.
1407/1407 [==============================] - 45s 32ms/step - loss: 0.2412 - 

1407/1407 [==============================] - ETA: 0s - loss: 0.2415 - accuracy: 0.9127
Epoch 00139: ReduceLROnPlateau reducing learning rate to 5.960464760645934e-11.
1407/1407 [==============================] - 45s 32ms/step - loss: 0.2415 - accuracy: 0.9127 - val_loss: 0.2929 - val_accuracy: 0.7530
Epoch 140/400
1407/1407 [==============================] - 45s 32ms/step - loss: 0.2415 - accuracy: 0.9133 - val_loss: 0.2930 - val_accuracy: 0.7516
Epoch 141/400
1407/1407 [==============================] - 45s 32ms/step - loss: 0.2414 - accuracy: 0.9125 - val_loss: 0.2930 - val_accuracy: 0.7520
Epoch 142/400
1407/1407 [==============================] - 45s 32ms/step - loss: 0.2416 - accuracy: 0.9114 - val_loss: 0.2929 - val_accuracy: 0.7506
Epoch 143/400
1407/1407 [==============================] - 45s 32ms/step - loss: 0.2415 - accuracy: 0.9122 - val_loss: 0.2932 - val_accuracy: 0.7514
Epoch 144/400
1407/1407 [==============================] - ETA: 0s - loss: 0.2413 - accuracy: 0.9128
E

1407/1407 [==============================] - 45s 32ms/step - loss: 0.2420 - accuracy: 0.9116 - val_loss: 0.2933 - val_accuracy: 0.7522
Epoch 184/400
1407/1407 [==============================] - ETA: 0s - loss: 0.2416 - accuracy: 0.9120
Epoch 00184: ReduceLROnPlateau reducing learning rate to 1.164153273563659e-13.
1407/1407 [==============================] - 45s 32ms/step - loss: 0.2416 - accuracy: 0.9120 - val_loss: 0.2932 - val_accuracy: 0.7528
Epoch 185/400
1407/1407 [==============================] - 45s 32ms/step - loss: 0.2421 - accuracy: 0.9093 - val_loss: 0.2934 - val_accuracy: 0.7520
Epoch 186/400
1407/1407 [==============================] - 45s 32ms/step - loss: 0.2419 - accuracy: 0.9118 - val_loss: 0.2929 - val_accuracy: 0.7522
Epoch 187/400
1407/1407 [==============================] - 45s 32ms/step - loss: 0.2415 - accuracy: 0.9120 - val_loss: 0.2929 - val_accuracy: 0.7518
Epoch 188/400
1407/1407 [==============================] - 45s 32ms/step - loss: 0.2414 - accuracy: 0.

1407/1407 [==============================] - 46s 32ms/step - loss: 0.2416 - accuracy: 0.9125 - val_loss: 0.2928 - val_accuracy: 0.7518
Epoch 228/400
1407/1407 [==============================] - 45s 32ms/step - loss: 0.2418 - accuracy: 0.9113 - val_loss: 0.2932 - val_accuracy: 0.7510
Epoch 229/400
1407/1407 [==============================] - ETA: 0s - loss: 0.2418 - accuracy: 0.9113
Epoch 00229: ReduceLROnPlateau reducing learning rate to 2.2737368624290214e-16.
1407/1407 [==============================] - 46s 33ms/step - loss: 0.2418 - accuracy: 0.9113 - val_loss: 0.2929 - val_accuracy: 0.7514
Epoch 230/400
1407/1407 [==============================] - 46s 32ms/step - loss: 0.2414 - accuracy: 0.9118 - val_loss: 0.2932 - val_accuracy: 0.7536
Epoch 231/400
1407/1407 [==============================] - 46s 32ms/step - loss: 0.2416 - accuracy: 0.9128 - val_loss: 0.2934 - val_accuracy: 0.7516
Epoch 232/400
1407/1407 [==============================] - 45s 32ms/step - loss: 0.2417 - accuracy: 0

1407/1407 [==============================] - 45s 32ms/step - loss: 0.2416 - accuracy: 0.9123 - val_loss: 0.2931 - val_accuracy: 0.7524
Epoch 272/400
1407/1407 [==============================] - 45s 32ms/step - loss: 0.2414 - accuracy: 0.9133 - val_loss: 0.2929 - val_accuracy: 0.7516
Epoch 273/400
1407/1407 [==============================] - 45s 32ms/step - loss: 0.2415 - accuracy: 0.9125 - val_loss: 0.2932 - val_accuracy: 0.7520
Epoch 274/400
1407/1407 [==============================] - ETA: 0s - loss: 0.2415 - accuracy: 0.9124
Epoch 00274: ReduceLROnPlateau reducing learning rate to 4.440892309431682e-19.
1407/1407 [==============================] - 45s 32ms/step - loss: 0.2415 - accuracy: 0.9124 - val_loss: 0.2931 - val_accuracy: 0.7508
Epoch 275/400
1407/1407 [==============================] - 45s 32ms/step - loss: 0.2416 - accuracy: 0.9123 - val_loss: 0.2930 - val_accuracy: 0.7536
Epoch 276/400
1407/1407 [==============================] - 45s 32ms/step - loss: 0.2414 - accuracy: 0.

1407/1407 [==============================] - 47s 33ms/step - loss: 0.2415 - accuracy: 0.9111 - val_loss: 0.2930 - val_accuracy: 0.7512
Epoch 316/400
1407/1407 [==============================] - 46s 33ms/step - loss: 0.2417 - accuracy: 0.9116 - val_loss: 0.2932 - val_accuracy: 0.7530
Epoch 317/400
1407/1407 [==============================] - 45s 32ms/step - loss: 0.2417 - accuracy: 0.9114 - val_loss: 0.2930 - val_accuracy: 0.7522
Epoch 318/400
1407/1407 [==============================] - 45s 32ms/step - loss: 0.2416 - accuracy: 0.9120 - val_loss: 0.2931 - val_accuracy: 0.7520
Epoch 319/400
1407/1407 [==============================] - ETA: 0s - loss: 0.2415 - accuracy: 0.9120
Epoch 00319: ReduceLROnPlateau reducing learning rate to 8.673617791858755e-22.
1407/1407 [==============================] - 45s 32ms/step - loss: 0.2415 - accuracy: 0.9120 - val_loss: 0.2931 - val_accuracy: 0.7522
Epoch 320/400
1407/1407 [==============================] - 45s 32ms/step - loss: 0.2416 - accuracy: 0.

Epoch 360/400
1407/1407 [==============================] - 45s 32ms/step - loss: 0.2414 - accuracy: 0.9124 - val_loss: 0.2931 - val_accuracy: 0.7530
Epoch 361/400
1407/1407 [==============================] - 45s 32ms/step - loss: 0.2413 - accuracy: 0.9138 - val_loss: 0.2933 - val_accuracy: 0.7520
Epoch 362/400
1407/1407 [==============================] - 45s 32ms/step - loss: 0.2417 - accuracy: 0.9115 - val_loss: 0.2929 - val_accuracy: 0.7522
Epoch 363/400
1407/1407 [==============================] - 45s 32ms/step - loss: 0.2417 - accuracy: 0.9121 - val_loss: 0.2931 - val_accuracy: 0.7514
Epoch 364/400
1407/1407 [==============================] - ETA: 0s - loss: 0.2416 - accuracy: 0.9118
Epoch 00364: ReduceLROnPlateau reducing learning rate to 1.694065974972413e-24.
1407/1407 [==============================] - 45s 32ms/step - loss: 0.2416 - accuracy: 0.9118 - val_loss: 0.2929 - val_accuracy: 0.7524
Epoch 365/400
1407/1407 [==============================] - 45s 32ms/step - loss: 0.2420 

In [ ]:
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss", patience=10,
        restore_best_weights=True
    ),